In [1]:
!pip install timer

In [2]:
import skimage
from matplotlib import pyplot as plt

from skimage import color
from skimage.color import rgb2gray

import cv2
import numpy as np
from scipy.ndimage import gaussian_filter
from skimage.filters import gabor_kernel
from scipy.signal import convolve2d
from scipy import misc
import os
import random
import glob
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from skimage.feature import graycomatrix, graycoprops
import pandas as pd
from timer import timer
import multiprocessing
from multiprocessing import Pool
from scipy.stats import kurtosis
import numpy as np
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
import glob
import tensorflow as tf
from sklearn.mixture import GaussianMixture


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Loading Images Function


In [3]:
def load_images(image_dir):
    # Define the directory containing the JPG images

    # Find all files in the directory that end with JPG
    #image_file_names = os.listdir(image_dir)
    image_paths =image_paths # [f"{image_dir}/{fname}" for fname in image_file_names if "jpg" in fname or "JPG" in fname]
    #image_paths = glob.glob(image_dir + '*.jpg')
    #image_paths.extend(glob.glob(image_dir + '*.JPG'))
    X=np.array([])
    X_color = np.array([])
    for image_path in image_paths:
        # Load the H&E slide image
        image=plt.imread(image_path)
        image=cv2.resize(image,(512,512))
        image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        # Normalize the grayscale image to have pixel values between 0 and 1
        image_norm = image_gray / np.max(image_gray)

        # Reshape the image into a 4D array for input into the autoencoder
        image_norm = np.reshape(image_norm, (1, image_norm.shape[0], image_norm.shape[1], 1))
        X = np.append(X,image_norm)
        X_color = np.append(X_color,image)
    X=np.vstack(X)
    return X, X_color, image_norm,image_paths


def load_image(fname,image_dir):
    image_path = f"{image_dir}/{fname}"
    image=plt.imread(image_path)
    image=cv2.resize(image,(512,512))
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Normalize the grayscale image to have pixel values between 0 and 1
    image_norm = image_gray / np.max(image_gray)

    # Reshape the image into a 4D array for input into the autoencoder
    image_norm = np.reshape(image_norm, (1, image_norm.shape[0], image_norm.shape[1], 1))
    image = np.reshape(image,(1,image.shape[0],image.shape[1],image.shape[2]))
    return image_norm, image


## Image Segmentation Code 

In [4]:
def extract_segmented_image(img,autoencoder, visualize=False):
    input_layer = autoencoder.get_layer("input_1").input
    encoded = autoencoder.get_layer("conv2d_2").output   
    encoder = Model(inputs=input_layer,outputs= encoded)
    encoded_img = encoder.predict(np.reshape(img, (1, img.shape[1],512, 1))) #data[index]
    t,h, w, d = encoded_img.shape
    encoded_img_flat = encoded_img.reshape(h*w, d)
    kmeans = KMeans(n_clusters=4).fit(encoded_img_flat)
    labels = kmeans.labels_
    segmented_img = labels.reshape(h, w)
    segmented_image = cv2.resize(segmented_img.astype('uint8'), (512, 512))
    if visualize:
        plt.figure()
        plt.imshow(segmented_image)
    return segmented_image,labels

def mask_retrieve(value,segmented_image,visualize=False):
    mask = (segmented_image == value).astype(np.uint8) * 255
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    if visualize:
        plt.figure()
        plt.imshow(mask, cmap='gray')
    return mask

In [5]:
def segment_image(gray,color,autoencoder,visualize=False):
    segmented_image,labels=extract_segmented_image(gray,autoencoder)     #(data,chosen,index)
    contour_definitions=[]
    color_r=[]
    results=[]
    masks_dict={}
    regions = ["Epithelium","bg1","Gland","bg2"]
    for i in np.unique(labels):  

        label=value=i
        mask_raw=mask = mask_retrieve(value,segmented_image)
        masks_dict[label]=mask
        contour_areas = []
        contour_lengths = []
        approx_functions = []
        contour_dim=[]
        contour_solidity=[]
        contour_circularity=[]
        contour_aspect_ratio=[]
        set_of_center_coords = []
        

        con, cb = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)

        cluster_mean_color = [np.mean(color[segmented_image == value], axis=0) for i in range(1)]

        color_r.append(cluster_mean_color[0][1])
        img_with_contours = np.copy(color) #np.copy(image)
        area_threshold=10
        aspect_ratio_threshold=10
        circularity_threshold=10

        for contour in con:
            area = cv2.contourArea(contour)
            if area > area_threshold:
                M = cv2.moments(contour)
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                center_coords = (cX,cY)
                set_of_center_coords.append(center_coords)
                contour_areas.append(area)
                perimeter = cv2.arcLength(contour, True)
                contour_lengths.append(perimeter)
                approx = cv2.approxPolyDP(contour, 0.01*perimeter, True)
                approx_functions.append(approx)
                contour_dim.append(contour)
                label = f"Contour {i+1}\nArea: {area:.2f}\nPerimeter: {perimeter:.2f}"
                x,y,w,h = cv2.boundingRect(approx)
                aspect_ratio = float(w)/h

                contour_aspect_ratio.append(aspect_ratio)
                
                circularity = (4*np.pi*area)/(perimeter**2)
                contour_circularity.append(circularity)
                
                
                solidity = area/cv2.contourArea(cv2.convexHull(contour))
                contour_solidity.append(solidity)
                
               
                # Classify the contour based on the shape descriptors
                if aspect_ratio >= aspect_ratio_threshold and circularity >= circularity_threshold:
                    shape = 'tubular/circular'
                else:
                    shape = 'branched'

                cv2.drawContours(img_with_contours, [contour], -1, (0, 0, 255), 3)
        if visualize:
            plt.figure()           
            plt.imshow(img_with_contours)

        for j, (area, length, approx,solidity,circularity,aspect_ratio,center_coords,contour) in enumerate(zip(contour_areas, contour_lengths, approx_functions,contour_solidity,contour_circularity,contour_aspect_ratio,set_of_center_coords,contour_dim)):
            result = {
                'contour_number': j+1,
                'area': area,
                'linear_dimension': length,
                'approximate_function': approx.tolist(),
                'class':value,
                'shape':shape,
                'mean_R_color':cluster_mean_color[0][1],
                'mask':mask,
                'mask_raw':mask_raw,
                'circularity':circularity,
                'solidity':solidity,
                'aspect_ratio':aspect_ratio,
                'center_coords':center_coords,
                'contour_dim': contour
            }
            results.append(result)
  
    return results,color_r,segmented_image,labels,masks_dict



In [6]:
def label_masks(results,color_r):
    for res in results:
        if res['mean_R_color']==max(color_r):
            res["region"]="Gland"
        else:
            res["region"]=""
            
    # Filter for class region  gland
    gland_contours = [result for result in results if result['region'] == 'Gland']
    unique_values = set()

    for my_dict in gland_contours:
        unique_values.add(my_dict['area'])

    top_five_values = sorted(unique_values, reverse=True)[:10]



    # Loop through gland contours
    for gland_contour in gland_contours:
    # print(gland_contour)
        
        if gland_contour['area'] in top_five_values:
                #print("here")
                gland_approx = np.array(gland_contour['approximate_function'])

                # Reshape gland_approx to match shape of other approx functions


                # Initialize closest distance and class
                closest_distance = np.inf
                closest_class = None

                # Loop through all contours to find closest class
                for result in results:
                    # Skip gland contours and contours with different shape
                    if result['region'] == 'Gland': # : or result['shape'] != gland_contour['shape']:
                        continue
            # Reshape gland_approx to match the shape of approx
                    approx = np.array(result['approximate_function'])
                    gland_approx_resized = np.resize(gland_approx, approx.shape)

                    # Calculate distance between approximate functions
                    distance = np.linalg.norm(gland_approx_resized - approx)

                    # Check if this is the closest class
                    if distance < closest_distance:
                        closest_distance = distance
                        closest_class = result['class']

                # Assign closest class to gland contour
                gland_contour['close_class'] = closest_class if closest_class is not None else 'black'
        else:
            gland_contour['close_class']=''


    unique_values = []

    for my_dict in gland_contours:
        unique_values.append(my_dict['close_class'])


    count_dict = {}

    for item in unique_values:
        if item in count_dict:
            count_dict[item] += 1
        else:
            count_dict[item] = 1

    sorted_counts = sorted(count_dict.items(), key=lambda x: x[1], reverse=True)

    second_max = sorted_counts[1] if len(sorted_counts) > 1 else sorted_counts[0]

    second_max[0]

    for res in results:
        if res["class"]==second_max[0]:
            res["region"]="Epithelium"

    unique_value = set()

    for my_dict in results:
        unique_value.add(my_dict['region'])
    return results

In [7]:

def reformat_results(results):
    reformatted_results = {}
    reformatted_results["Epithelium"] = {
        "analyzed_contours":[],
        "mask":[]
    }
    reformatted_results["Stroma"] = {
        "analyzed_contours":[],
        "mask":[]
    }
    reformatted_results["Gland"] = {
        "analyzed_contours":[],
        "mask":[]
    }


    for result in results: 
        current_result = result.copy()
        if result["region"]=='' or result["region"]=="":
            if len(reformatted_results["Stroma"]["mask"]) == 0:
                reformatted_results["Stroma"]["mask"] = result["mask"]
            reformatted_results["Stroma"]["analyzed_contours"].append(current_result)
        else:
            region = result["region"]

            if result["region"]=='Epithelium':
                  reformatted_results[region]["mask"] = result["mask_raw"]               
            elif len(reformatted_results[region]["mask"]) == 0:
                  reformatted_results[region]["mask"] = result["mask"]
            
            reformatted_results[region]["analyzed_contours"].append(current_result)
    return reformatted_results


In [8]:
from skimage.filters import threshold_multiotsu


def transform_encoded_prediction_to_image_space(segmentation_mask):
    min_val = np.amin(segmentation_mask)
    max_val = np.amax(segmentation_mask)
    diff = max_val - min_val
    return 255*(segmentation_mask - min_val)/ diff



In [12]:
def grow_epi(mask):
    #mask=
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    #mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return mask

## Feature Extraction Code

In [ ]:
def apply_mask(img_color,mask):
    masked_img = np.zeros((np.shape(img_color)[1],np.shape(img_color)[2],3))
    for i in range(3):
        masked_img[:,:,i] = img_color[0,:,:,i]*mask
    masked_img = masked_img.astype("uint8")
    return masked_img

def k_means_segmentation(img_color):
    img = cv2.cvtColor(img_color,cv2.COLOR_BGR2RGB)
    twoDimage = img.reshape((-1,3))
    twoDimage = np.float32(twoDimage)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 3
    attempts=10
    ret,label,center=cv2.kmeans(twoDimage,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    result_image = res.reshape((img.shape))
    return result_image

def return_nuclei_mask(img_color, K=3, attempts = 10):
    # Kmeans
    img = cv2.cvtColor(img_color,cv2.COLOR_BGR2RGB)
    twoDimage = img.reshape((-1,3))
    twoDimage = np.float32(twoDimage)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret,label,center=cv2.kmeans(twoDimage,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
    # collect masks corresponding to each label
    masks = []
    # the masks with 0 RGB is background & largest RGB value is the nuclei
    rgb_sum_max = 0
    mask_nuclei = []
    for k in range(K):
        m = (label == k)
        m = m.reshape((img.shape[0], img.shape[1]))
        img_test = np.copy(img)
        for i in range(3):
            nuclei_pixels = img_test[:,:,i]*m
            img_test[:,:,i] = nuclei_pixels
        rgb_sum = sum(sum(sum(img_test)))
        if (rgb_sum > rgb_sum_max):
            mask_nuclei = m
            rgb_sum_max = rgb_sum
    return mask_nuclei



def segment_nuclei(img_color,epithelium_mask,visualize=False):
    mask = (epithelium_mask==255) # results["epithelium"]["mask"]
    masked_img = apply_mask(img_color,mask)
    nuclei_mask = return_nuclei_mask(masked_img, K=3, attempts = 10)

    nuclei_masked_img = apply_mask(img_color,nuclei_mask)

    if visualize:
        plt.figure()
        plt.imshow(img_color[0,:,:,:])
        plt.axis('off')
        plt.title("Raw Image")
        plt.figure()
        plt.title("Epithelium Masked Image")
        plt.imshow(masked_img)
        plt.axis('off')
        plt.figure()
        plt.axis('off')
        plt.title("Nuclei Segmented Image")
        plt.imshow(nuclei_masked_img)
        plt.axis('off')

    return nuclei_mask,nuclei_masked_img

def fit_ellipses_to_nuclei(img_color,nuclei_masked_img,area_exclusion_percentile = 80,draw_contours=False,draw_ellipses = False):
    nuclei_masked_img = cv2.cvtColor(nuclei_masked_img, cv2.COLOR_BGR2GRAY)

    con, cb = cv2.findContours(nuclei_masked_img, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    
    if draw_contours:
        img_with_contours = np.copy(img_color)
    if draw_ellipses:
        color=1 #ellipse color
        img_with_ellipses = np.copy(img_color)

    ellipses = []
    ellipse_areas = []
    approx_functions = []
    contour_areas = []

    for contour in con:
        # Can't fit an ellipse if less than 5 points in contour
        if len(contour) >5:
            area = cv2.contourArea(contour)
            contour_areas.append(area)
            minEllipse = cv2.fitEllipse(contour)
            ellipses.append(minEllipse)
            area_ellipse = minEllipse[1][0]*minEllipse[1][1]*np.pi
            ellipse_areas.append(area)
            perimeter = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.01*perimeter, True)
            approx_functions.append(approx)
  
    area_exclusion = np.percentile(ellipse_areas,area_exclusion_percentile)
    ellipses= [ellipses[i] for i,area in enumerate(ellipse_areas) if area <area_exclusion ] 
    approx_functions= [approx_functions[i] for i,area in enumerate(ellipse_areas) if area <area_exclusion ] #Exclu
    contour_areas= [contour_areas[i] for i,area in enumerate(ellipse_areas) if area <area_exclusion ] #Exclu

    if draw_contours:
        for contour in con: 
            cv2.drawContours(img_with_contours, [contour], -1, (0, 0, 255), 1)
    if draw_ellipses:
        for ellipse in ellipses:
            cv2.ellipse(img_with_ellipses, ellipse, color, 1)


    if draw_contours and draw_ellipses:
        plt.figure()
        plt.imshow(img_with_contours)
        plt.axis('off')
        plt.figure()
        plt.imshow(img_with_ellipses)
        plt.axis('off')

    elif draw_ellipses:
        plt.figure()
        plt.imshow(img_with_ellipses)
        plt.axis('off')

    elif draw_contours:
        plt.figure()
        plt.imshow(img_with_contours)
        plt.axis('off')

    return con,ellipses,approx_functions,contour_areas

def calculate_nuclei_gland_angle_difference(gland_center,nuclei_center,nuclei_angle):
    del_x = gland_center[0] - nuclei_center[0]
    del_y = (gland_center[1] - nuclei_center[1])
    theta = 180 - np.pi*np.arctan(del_x/del_y)/180 # between gland to nuclei vector and vertical line going clockwise
    angle_diff= theta - nuclei_angle
    if angle_diff > 90:
        angle_diff = 180 - angle_diff
    return angle_diff,theta


def find_gland_index(cell_coord,gland_contours):
    # cell_coord: 2x1 touple of the ellipse contour center of the cell(coordinate_x, coordinate_y)
    # gland_contours: all gland contours (e.g. results["gland"]["raw_contours"])
    dist = [] # collect all distances from the cell to all glands
    for gc in gland_contours:
        d = cv2.pointPolygonTest(gc,cell_coord,True)
        dist.append(d)
    index_gland = np.argmin(dist)
    # return index of the gland that the cell belongs to 
    return index_gland

def calculate_nuclei_features(nuclei_ellipses,gland_contours,analyzed_gland_contours):
    #nuclei rotation, gland to nuclei angle, difference
    
    n_nuclei = len(nuclei_ellipses)
    angles = np.zeros((n_nuclei,3))

    for nucleus_i in range(n_nuclei):
      try:
        nucleus_coords = nuclei_ellipses[nucleus_i][0]
        nucleus_angle = nuclei_ellipses[nucleus_i][2]
        nearest_gland_index = find_gland_index(nucleus_coords,gland_contours)
        nearest_gland_center =analyzed_gland_contours[nearest_gland_index]['center_coords']
        angle_diff,gland_to_nucleus_angle = calculate_nuclei_gland_angle_difference(nearest_gland_center,nucleus_coords,nucleus_angle)
        angles[nucleus_i,:]=[nucleus_angle,gland_to_nucleus_angle,angle_diff]
      except:
        continue
    return angles

def calculate_gland_overlap(analyzed_glands,color_img,visualize=False):
    overlap_percentages = np.zeros(len(analyzed_glands))
    contour_img = np.copy(color_img)
    for i,result in enumerate(analyzed_glands):
        contour = np.array(result['approximate_function'], dtype=np.int32)

        (x,y),radius = cv2.minEnclosingCircle(np.array(result['approximate_function']))
        center = (int(x),int(y))
        area=result['area']
        radius = int(np.sqrt(area/np.pi))
        cv2.circle(contour_img,center,radius,(0,255,0),2)
        cv2.drawContours(contour_img, [contour], 0, (0, 255, 0), 2)

        mask = np.zeros_like(contour_img)
        cv2.fillPoly(mask, [contour], (255, 255, 255)) # fill the contour with a color

        circle_mask = cv2.circle(np.zeros_like(contour_img), center, radius, (255, 255, 255), -1) #fill the circle with same color
        intersection = cv2.bitwise_and(mask, circle_mask) #intersection between two masks
        
        overlap_percentage = np.sum(intersection) / np.sum(mask) * 100 # compute the percentage of overlap
        overlap_percentages[i]= overlap_percentage
        # print the overlap percentage
        #print(f"Overlap percentage: {overlap_percentage:.2f}%")

    if visualize:
        plt.figure()
        plt.imshow(contour_img)
        plt.axis('off')

    return overlap_percentages

def remove_blood(img_color,visualize=False,neg_log_likelihood_thresh = -10):
    #Defining location of blood in 2D red blue color space
    rough_red_threshold = 200
    rough_blue_threshold = 150

    #Defining copies of image to manipulate in a flattened state
    im_to_slice= copy_im.reshape((512*512,3))
    full_im = copy_im.reshape((512*512,3))

    #Slicing image to notch out area of blood color
    sliced_im = im_to_slice[(im_to_slice[:,0]>rough_red_threshold)&(im_to_slice[:,2]<rough_blue_threshold),:]
    if np.shape(sliced_im)[0] < 10: #If less than 10 blood like pixels do nothing
        return img_color
    #Formatting to GMM compatible data and fitting single gmm to data
    X = np.vstack([sliced_im[:,0],sliced_im[:,2]])
    X = X.T
    means_init = np.array([[150,175]])
    gm = GaussianMixture(n_components=1, random_state=0,means_init=means_init).fit(X)

    #Scoring every pixel in full image
    X_full =  np.vstack([full_im[:,0],full_im[:,2]])
    X_full = X_full.T
    cluster_preds_full =gm.score_samples(X_full)

    #Removing pixels that are in the red space based on a likelihood score
    
    if visualize:
        #Visualizing removed pixels 
        r_slice_full = full_im[:,0]
        b_slice_full = full_im[:,2]
        plt.figure()
        plt.plot(r_slice_full[cluster_preds_full > neg_log_likelihood_thresh], b_slice_full[cluster_preds_full > neg_log_likelihood_thresh],'r.',markersize=1)
        plt.plot(r_slice_full[cluster_preds_full < neg_log_likelihood_thresh], b_slice_full[cluster_preds_full < neg_log_likelihood_thresh],'b.',markersize=1)
        plt.axis('off')

    full_im[cluster_preds_full > neg_log_likelihood_thresh,:]=215
    proper_full_im = full_im.reshape((512,512,3))
    
    if visualize:

        #Visualizing Final image
        plt.figure()
        plt.imshow(proper_full_im)
        plt.axis('off')

    return proper_full_im
        

def nuclei_color_extraction(img_color,ellipses):
    # STD value storage
    nuclei_std = []
    nuclei_mean = []
    for ellipse in ellipses:
        # Create a binary mask with the same size as the input image
        mask = np.zeros_like(img_color[:,:,:])
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        # Find the center and axes of ellipse
        a, b = ellipse[1]
        cx, cy = ellipse[0]
        # Fill the ellipse with white color
        cv2.ellipse(mask, (int(cx), int(cy)), (int(a), int(b)), 0, 0, 360, (255, 255, 255), -1)
        # Apply the filled ellipse mask to original image and calculate std within the mask 
        mean_color,std_color = cv2.meanStdDev(img_color, mask=mask)
        # change (3,1) to (3,)
        std_r, std_g, std_b = std_color.squeeze()
        # use root mean square to combine the std from R, G, and B channels 
        total_std = np.sqrt((std_r ** 2 + std_g ** 2 + std_b ** 2) / 3)
        nuclei_std.append(total_std)
        nuclei_mean.append(mean_color)
    return nuclei_std,nuclei_mean

def find_all_gland_indexes_and_distances(ellipses,gland_contours):
        # calculate and return the glands all nucleis belong to and the distances to those glands
        gland_indexes = []
        gland_nuclei_distances = []
        for Ellipse in ellipses:
          cell_coord = Ellipse[0]
          dist = [] # collect all distances from the cell to all glands
          for gc in gland_contours:
            d = cv2.pointPolygonTest(gc,cell_coord,True)
            dist.append(d)
          index_gland = np.argmin(dist)
          gland_nuclei_distance = dist[index_gland]
          gland_nuclei_distances.append(gland_nuclei_distance)
          gland_indexes.append(index_gland)
        gland_indexes = np.array(gland_indexes)
        gland_nuclei_distances = np.array(gland_nuclei_distances)
        return gland_indexes,gland_nuclei_distances 

def find_nuclei_to_gland_std (ellipses,gland_contours,std_threshold = 0.2):
      gland_indexes,gland_nuclei_distances = find_all_gland_indexes_and_distances(ellipses,gland_contours)
      # get unqiue gland indexes
      glands = np.unique(gland_indexes)
      # store std of distance
      std_distances = []
      for gland in glands:
        # get std of distances of all nucleis belong to one gland
        mask_g = (gland_indexes==gland)
        # normalize the std with mi-max range of distance
        std_distance = np.std(gland_nuclei_distances[mask_g]) / (np.max(gland_nuclei_distances)-np.min(gland_nuclei_distances))
        std_distances.append(std_distance)
      # count the number of glands (n_abnormal) with a gland-nuclei-distance std >= std_threshold
      std_distances = np.array(std_distances)
      n_abnormal = sum((std_distances >= std_threshold))
      # get the proportion of glands with a gland-nuclei-distance std >= std_threshold
      abnormal_proportion = n_abnormal / len(glands)
      return abnormal_proportion

def calculate_nuclei_overlap(approx_functions,nuclei_areas,color_img,visualize=False):
    overlap_percentages = np.zeros(len(approx_functions))
    contour_img = np.copy(color_img)
    for i,approx in enumerate(approx_functions):
        contour = np.array(approx, dtype=np.int32)

        (x,y),radius = cv2.minEnclosingCircle(np.array(approx))
        center = (int(x),int(y))
        area=nuclei_areas[i]
        radius = int(np.sqrt(area/np.pi))
        cv2.circle(contour_img,center,radius,(0,255,0),2)
        cv2.drawContours(contour_img, [contour], 0, (0, 255, 0), 2)

        mask = np.zeros_like(contour_img)
        cv2.fillPoly(mask, [contour], (255, 255, 255)) # fill the contour with a color

        circle_mask = cv2.circle(np.zeros_like(contour_img), center, radius, (255, 255, 255), -1) #fill the circle with same color
        intersection = cv2.bitwise_and(mask, circle_mask) #intersection between two masks
        
        overlap_percentage = np.sum(intersection) / np.sum(mask) * 100 # compute the percentage of overlap
        overlap_percentages[i]= overlap_percentage
        # print the overlap percentage
        #print(f"Overlap percentage: {overlap_percentage:.2f}%")

    if visualize:
        plt.figure()
        plt.imshow(contour_img)
        plt.axis('off')
    
    return overlap_percentages

# Pipeline

A small tes code to debug the modules please skip to the step of running 

In [14]:
model_path = "dataset_2/autoenco_glasshour_v2_1channel/model/"#"/content/drive/My Drive/ECE613/autoenco_glasshour_v2_1channel/"     # #Change for yours
autoencoder = tf.keras.models.load_model(model_path) #Model(input_img, decoded)
autoencoder.summary()

image_dir ='./mini dataset/dataset_2/' #'./sample_input_H_E/' #

# Find all files in the directory that end with JPG
image_paths = glob.glob(image_dir + '*.jpg')
image_paths.extend(glob.glob(image_dir + '*.JPG'))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512, 512, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 512, 512, 128)     1280      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 256, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 32)      18464 

In [ ]:
#feature_df = pd.DataFrame(columns =headers)

image_dir = "/content/drive/My Drive/ECE613/Model_eval_dataset_03042023/"  
#autoencoder=
fnames = image_paths  # os.listdir(image_dir)
fnames =[fname for fname in fnames if "jpg" in fname or "JPG" in fname]
fname=fnames[100]
#for fname in fnames:
features = {}
img, img_color = load_image(fname,image_dir)
segmentation_results,color_r,segmented_image,labels,masks_dict = segment_image(img,img_color[0,:,:,:],autoencoder,visualize=False)

segmentation_results = label_masks(segmentation_results,color_r)

segmentation_results = reformat_results(segmentation_results)

In [ ]:
plt.imshow(epithelium_mask)

In [ ]:
def run_pipeline(image_dir,headers,autoencoder,visualize=False):

    feature_df = pd.DataFrame(columns =headers)
    fnames = os.listdir(image_dir)
    fnames =[fname for fname in fnames if "jpg" in fname or "JPG" in fname]
    #fnames=fnames[0]
    for fname in fnames:
        features = {}
        img, img_color = load_image(fname,image_dir)
        segmentation_results,color_r,segmented_image,labels,masks_dict = segment_image(img,img_color[0,:,:,:],autoencoder,visualize=False)

        segmentation_results = label_masks(segmentation_results,color_r)

        segmentation_results = reformat_results(segmentation_results)


        epithelium_mask = grow_epi(segmentation_results["Epithelium"]["mask"])

        nuclei_mask,nuclei_masked_img = segment_nuclei(
            img_color,
            epithelium_mask,
            visualize=visualize
        )

        nuclei_contours,nuclei_ellipses,nuclei_approx_functions,nuclei_contour_areas = fit_ellipses_to_nuclei(
            img_color[0,:,:,:],
            nuclei_masked_img,
            draw_contours=visualize,
            draw_ellipses = visualize,
            area_exclusion_percentile = 85
        )

        analyzed_gland_contours = segmentation_results["Gland"]['analyzed_contours']

        gland_areas = [gland["area"] for gland in analyzed_gland_contours]

        gland_contours =  [segmentation_results["Gland"]['analyzed_contours'][i]["contour_dim"] for i in range(len(segmentation_results["Gland"]['analyzed_contours']))]

        angles = calculate_nuclei_features(
            nuclei_ellipses,
            gland_contours,
            analyzed_gland_contours
        )
       
        gland_overlap_percentages = calculate_gland_overlap(
            segmentation_results["Gland"]['analyzed_contours'],
            img_color[0,:,:,:],
        )

        nuclei_overlap_percentages = calculate_nuclei_overlap(
            nuclei_approx_functions,
            nuclei_contour_areas,
            img_color[0,:,:,:],
            visualize=visualize
        )

        pct_glands_abnormally_spaced_nuclei = find_nuclei_to_gland_std (
            nuclei_ellipses,
            gland_contours,
            std_threshold = 0.2
        )

        nuclei_color_std,nuclei_mean_color = nuclei_color_extraction(
            img_color[0,:,:,:],
            nuclei_ellipses
        )

        gland_solidity = [segmentation_results["Gland"]['analyzed_contours'][i]["solidity"] for i in range(len(segmentation_results["Gland"]['analyzed_contours']))]
        gland_solidity_median = np.median(gland_solidity)
        gland_solidity_std = np.std(gland_solidity)

        gland_circularity = [segmentation_results["Gland"]['analyzed_contours'][i]["circularity"] for i in range(len(segmentation_results["Gland"]['analyzed_contours']))]
        gland_circularity_median = np.median(gland_circularity)
        gland_circularity_std = np.std(gland_circularity)

        gland_aspect_ratio= [segmentation_results["Gland"]['analyzed_contours'][i]["aspect_ratio"] for i in range(len(segmentation_results["Gland"]['analyzed_contours']))]
        gland_aspect_ratio_median = np.median(gland_aspect_ratio)
        gland_aspect_ratio_std = np.std(gland_aspect_ratio)

        features["file_paths"] = fname
        features['nuclei_angle_diff_mean'],features['nuclei_angle_mean'],features['gland_to_nuclei_angle_mean'] = np.mean(angles,axis=0)
        features['nuclei_angle_diff_std'] =  np.std(angles[0,:],axis=0)
        features['gland_overlap_percentage_mean'] = np.mean(gland_overlap_percentages)
        features['gland_overlap_percentage_std'] = np.std(gland_overlap_percentages)
        features['nuclei_overlap_percentage_mean'] = np.mean(nuclei_overlap_percentages)
        features['nuclei_overlap_percentage_std'] = np.std(nuclei_overlap_percentages)
        features["percent_glands_with_abnormally_spaced_nuclei"] = pct_glands_abnormally_spaced_nuclei
        features["nuclei_color_std"] = np.mean(nuclei_color_std,axis=0)

        nuclei_color_mean_r,nuclei_color_mean_g,nuclei_color_mean_b=np.mean(nuclei_mean_color,axis=0)
        features["nuclei_color_mean_r"] = nuclei_color_mean_r[0]
        features["nuclei_color_mean_g"] = nuclei_color_mean_g[0]
        features["nuclei_color_mean_b"] = nuclei_color_mean_b[0]

        features["gland_area_mean"] = np.mean(gland_areas)
        features["gland_area_std"] = np.std(gland_areas)
        features["gland_area_median"] = np.median(gland_areas)
        features["number_of_glands"] = len(gland_contours)

        features["gland_solidity_median"] = gland_solidity_median
        features["gland_solidity_std"] = gland_solidity_std

        features["gland_circularity_median"] = gland_solidity_median
        features["gland_circularity_std"] = gland_solidity_std

        features["gland_aspect_ratio_median"] = gland_solidity_median
        features["gland_aspect_ratio_std"] = gland_solidity_std
        feature_df.loc[len(feature_df)] = features
    return feature_df


In [ ]:
headers = [
    "file_paths", #added
    "gland_overlap_percentage_mean", #added
    "gland_overlap_percentage_std", #added
    "nuclei_overlap_percentage_mean",#added
    "nuclei_overlap_percentage_std", #added
 	"nuclei_angle_diff_mean", #added
    "nuclei_angle_diff_std", #added
 	"nuclei_angle_mean", #added
    "gland_to_nuclei_angle_mean", #added
    "nuclei_color_std", #added
 	"nuclei_color_mean_r",#added
 	"nuclei_color_mean_g",#added     
 	"nuclei_color_mean_b",#added
 	"number_of_blood_pixels_removed",
 	"blood_vessel_thickness",
 	"percent_glands_with_abnormally_spaced_nuclei", #added
    "number_of_glands",
 	"gland_area_mean", #added
 	"gland_area_std", #added
 	"gland_area_median", #added
    "stroma_to_gland_ratio",
    "shape_of_gland", #Count shape as circular, elliptical or branched
    "gland_solidity_median", #added
    "gland_solidity_std",#added
    "gland_circularity_median",#added
    "gland_circularity_std", #added
    "gland_aspect_ratio_median", #added
    "gland_aspect_ratio_std" #added
]

In [ ]:
image_dir = "/content/drive/My Drive/ECE613/Model_eval_dataset_03042023/"  
#  #"./drive/MyDrive/Waterloo/4B/ECE417/Test_Images_150"
model_path =  "/content/drive/My Drive/ECE613/autoenco_glasshour_v2_1channel/"   
autoencoder = tf.keras.models.load_model(model_path) #Model(input_img, decoded)




In [ ]:
feature_df  = run_pipeline(image_dir,headers,autoencoder,visualize=False)
display(feature_df)

In [ ]:
feature_df.to_csv("features.csv")

# Classifier

In [ ]:
feature_df_no_nan = feature_df.dropna(axis='columns')
for index, row in feature_df_no_nan.iterrows():
    feature_df_no_nan['file_paths'][index] = row['file_paths'][0:2]

In [ ]:
feature_df_no_nan['file_paths'].unique()

In [ ]:
for index, row in feature_df_no_nan.iterrows():
    if (feature_df_no_nan['file_paths'][index] == 'NE'):
        feature_df_no_nan['file_paths'][index] = 0
    elif (feature_df_no_nan['file_paths'][index] == 'EA'):
        feature_df_no_nan['file_paths'][index] = 1
    elif (feature_df_no_nan['file_paths'][index] == 'EH'):
        feature_df_no_nan['file_paths'][index] = 2
    elif (feature_df_no_nan['file_paths'][index] == 'EP'):
        feature_df_no_nan['file_paths'][index] = 3

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.model_selection import train_test_split
# separate the label and data
X = feature_df_no_nan.drop('file_paths', axis=1).astype(float)
y = feature_df_no_nan['file_paths'].astype(float)
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# create the decision tree classifier
clf = DecisionTreeClassifier()
# train the classifier on the training set
clf.fit(X_train, y_train)
# test the classifier on the testing set
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

y_pred = clf.predict(X_test)
class_lables =["NE", "EA", "EH","EP"]
cm = confusion_matrix(y_test, y_pred,labels=[0,1,2,3])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=class_lables)
disp.plot()